[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/nekrut/bda/blob/colab/lectures/lecture3.ipynb)

# Lecture 3: Data analysis ecosystem

Language + Dataframes + Viz

In this lecture, we'll simulate [Sanger sequencing](https://en.wikipedia.org/wiki/Sanger_sequencing) using Python. Along the way, we'll learn fundamental programming concepts—each biological step maps to a programming technique:

![](https://github.com/nekrut/bda/blob/colab/images/sanger-code-parallel.svg?raw=1)

First we import a module called [`random`](https://docs.python.org/3/library/random.html) which contains a number of functions for generating and working with random numbers

In [1]:
import random

## Generate a random sequence

Next, we will write a simple loop that generates a sequence of a preset length:

In [2]:
seq = ''
for _ in range(100):
    seq += random.choice('ATCG')

In [3]:
seq

'GACCTTCGCAAATGCGTCCTGTGACTGGTAGCCCACGCACCGGATTGAAACCAGTCGCCAGGAGGAACGAGATTCAACCTACTATATATTGTACCAATTC'

## Simulate one polymerase molecule

The code below iterates through each nucleotide in `seq`. When it encounters an 'A', it generates a random number between 0 and 1 using `random.random()`. If this number is smaller or equal than 0.5, the polymerase "terminates"—it adds a lowercase 'a' (representing incorporation of a ddNTP) to `synthesized_strand` and stops synthesis by breaking out of the loop. If the random number is greater than 0.5, synthesis continues normally.

For all other nucleotides (and for 'A' when synthesis continues), the uppercase nucleotide is added to `synthesized_strand`.

This simulates Sanger sequencing, where ddNTPs randomly terminate DNA synthesis at positions matching a specific base.

In [4]:
synthesized_strand = ''

for nucleotide in seq:
    if nucleotide == 'A':
        d_or_dd = random.random()
        if d_or_dd > 0.5:
            None
        else:
            synthesized_strand += 'a'
            break
    synthesized_strand += nucleotide

This can be simplified by first removing `d_or_dd` variable:

In [5]:
synthesized_strand = ''
for nucleotide in seq:
    if nucleotide == 'A':
        if random.random() > 0.5:
            None
        else:
            synthesized_strand += 'a'
            break
    synthesized_strand += nucleotide
print(synthesized_strand)

Ga


and removing unnecessary group of `if ... else` statements:

In [6]:
synthesized_strand = ''
for nucleotide in seq:
    if nucleotide == 'A' and random.random() > 0.5:
        synthesized_strand += 'a'
        break
    synthesized_strand += nucleotide
print(synthesized_strand)

Ga


finally let's make `synthesized_strand += 'a'` a bit more generic:

In [7]:
synthesized_strand = ''
for nucleotide in seq:
    if nucleotide == 'A' and random.random() > 0.5:
        synthesized_strand += nucleotide.lower()
        break
    synthesized_strand += nucleotide
print(synthesized_strand)

GACCTTCGCa


## Simulating multiple molecules

To simulate 10 polymerase molecules, we simply wrap the code from above into a `for` loop:

In [8]:
for _ in range(10):
    synthesized_strand = ''
    for nucleotide in seq:
        if nucleotide == 'A' and random.random() > 0.5:
            synthesized_strand += nucleotide.lower()
            break
        synthesized_strand += nucleotide
    print(synthesized_strand)

Ga
Ga
Ga
Ga
GACCTTCGCa
Ga
GACCTTCGCa
GACCTTCGCa
Ga
Ga


One problem with this code is that it does not actually save the newly synthesized strand: it simply prints it. To fix this we will create a [list](https://colab.research.google.com/github/AllenDowney/ThinkPython/blob/v3/chapters/chap09.ipynb) (or an array) called `new_strands` and initialize it by assigning an empty list to it:

In [9]:
new_strands = []
for _ in range(10):
    synthesized_strand = ''
    for nucleotide in seq:
        if nucleotide == 'A' and random.random() > 0.5:
            synthesized_strand += nucleotide.lower()
            break
        synthesized_strand += nucleotide
    new_strands.append(synthesized_strand)

In [10]:
new_strands

['GACCTTCGCAAa',
 'Ga',
 'Ga',
 'GACCTTCGCAAATGCGTCCTGTGACTGGTAGCCCa',
 'Ga',
 'Ga',
 'GACCTTCGCAAa',
 'GACCTTCGCa',
 'Ga',
 'GACCTTCGCa']

## Simulating multiple molecules and all nucleotides

And to repeat this for the remaining three nucleotides we will do the following crazy thing:

In [11]:
new_strands = []
for _ in range(10):
    synthesized_strand = ''
    for nucleotide in seq:
        if nucleotide == 'A' and random.random() > 0.5:
            synthesized_strand += nucleotide.lower()
            break
        synthesized_strand += nucleotide
    new_strands.append(synthesized_strand)

for _ in range(10):
    synthesized_strand = ''
    for nucleotide in seq:
        if nucleotide == 'C' and random.random() > 0.5:
            synthesized_strand += nucleotide.lower()
            break
        synthesized_strand += nucleotide
    new_strands.append(synthesized_strand)

for _ in range(10):
    synthesized_strand = ''
    for nucleotide in seq:
        if nucleotide == 'G' and random.random() > 0.5:
            synthesized_strand += nucleotide.lower()
            break
        synthesized_strand += nucleotide
    new_strands.append(synthesized_strand)

for _ in range(10):
    synthesized_strand = ''
    for nucleotide in seq:
        if nucleotide == 'T' and random.random() > 0.5:
            synthesized_strand += nucleotide.lower()
            break
        synthesized_strand += nucleotide
    new_strands.append(synthesized_strand)

In [12]:
len(new_strands)

40

Repeating the same code four times is just plain stupid, so instead we will write a function called `ddN`. Here we need to understand the scope of variables. The scope of a variable refers to the regions of the code where the variable can be accessed or modified. Variables that are defined within a certain block of code (such as a function) are said to have a *local* scope, meaning that they can only be accessed within that block of code. Variables that are defined outside of any function are said to have a *global* scope, meaning that they can be accessed from anywhere in the code.

In most programming languages, a variable defined within a function has a local scope and can only be accessed within that function. If a variable with the same name is defined outside the function, it will have a global scope. However, if a variable with the same name is defined within the function, it will take precedence over the global variable.

In Python, variables defined in the main module have global scope and can be accessed from any function or module. Variables defined within a function have local scope and can only be accessed within that function. **Unlike some other languages (like C++ or Java), Python does NOT have block scope**—variables defined within a loop or if statement remain accessible after the block ends.

In [13]:
def ddN(number_of_iterations, template, base, ddN_ratio):
    new_strands = []
    for _ in range(number_of_iterations):
        synthesized_strand = ''
        for nucleotide in template:
            if nucleotide == base and random.random() > ddN_ratio:
                synthesized_strand += nucleotide.lower()
                break
            synthesized_strand += nucleotide
        new_strands.append(synthesized_strand)
    return(new_strands)

In [14]:
ddN(10,seq,'A',0.5)

['GACCTTCGCAa',
 'Ga',
 'GACCTTCGCa',
 'Ga',
 'Ga',
 'Ga',
 'Ga',
 'GACCTTCGCa',
 'GACCTTCGCa',
 'GACCTTCGCAa']

To execute this function on all four types of ddNTPs, we need to wrap it in a `for` loop iterating over the four possibilities:

In [15]:
for nt in 'ATCG':
    ddN(10,seq,nt,0.5)

## A bit about [lists](https://colab.research.google.com/github/AllenDowney/ThinkPython/blob/v3/chapters/chap09.ipynb)
A list is like a container that can hold multiple items (elements).
- the first element will always be 0

To store the sequences being generated in the previous loop we will create and initialize a list called `seq_run`:

In [16]:
seq_run = []
for nt in 'ATCG':
    seq_run.append(ddN(10,seq,nt,0.5))

you will see that the seq run is a two-dimensional list:

In [17]:
seq_run

[['GACCTTCGCAa',
  'Ga',
  'GACCTTCGCa',
  'GACCTTCGCAAATGCGTCCTGTGa',
  'GACCTTCGCAa',
  'Ga',
  'Ga',
  'Ga',
  'Ga',
  'Ga'],
 ['GACCTt',
  'GACCt',
  'GACCt',
  'GACCt',
  'GACCTt',
  'GACCTt',
  'GACCt',
  'GACCt',
  'GACCTt',
  'GACCTt'],
 ['GAc',
  'GAc',
  'GACCTTCGc',
  'GACc',
  'GAc',
  'GACCTTc',
  'GAc',
  'GACc',
  'GAc',
  'GAc'],
 ['GACCTTCg',
  'g',
  'g',
  'GACCTTCg',
  'g',
  'g',
  'g',
  'GACCTTCGCAAATg',
  'GACCTTCGCAAATg',
  'g']]

as you will read in your next home assignment list elements can be addressed by "index". The first element has number 0:

In [18]:
seq_run[0]

['GACCTTCGCAa',
 'Ga',
 'GACCTTCGCa',
 'GACCTTCGCAAATGCGTCCTGTGa',
 'GACCTTCGCAa',
 'Ga',
 'Ga',
 'Ga',
 'Ga',
 'Ga']

## A bit about [dictionaries](https://colab.research.google.com/github/AllenDowney/ThinkPython/blob/v3/chapters/chap10.ipynb)

Another way to store these data is in a dictionary, which is a collection of key:value pairs where a key and value can be anything:

In [19]:
seq_run = {}
for nt in 'ATCG':
    seq_run[nt] = ddN(10,seq,nt,0.90)

In [20]:
seq_run

{'A': ['GACCTTCGCAAa',
  'GACCTTCGCAAATGCGTCCTGTGACTGGTAGCCCa',
  'Ga',
  'GACCTTCGCAAATGCGTCCTGTGACTGGTa',
  'GACCTTCGCAAATGCGTCCTGTGACTGGTAGCCCACGCACCGGATTGa',
  'GACCTTCGCAAATGCGTCCTGTGACTGGTAGCCCACGCACCGGATTGAAACCAGTCGCCAGGAGGAACGa',
  'GACCTTCGCa',
  'GACCTTCGCAa',
  'Ga',
  'Ga'],
 'T': ['GACCt',
  'GACCTTCGCAAATGCGTCCTGt',
  'GACCTTCGCAAATGCGTCCTGTGACt',
  'GACCTTCGCAAATGCGTCCTGt',
  'GACCTTCGCAAATGCGTCCTGTGACTGGTAGCCCACGCACCGGATTGAAACCAGTCGCCAGGAGGAACGAGATTCAACCTACTATATATTGTACCAATTC',
  'GACCTTCGCAAAt',
  'GACCTt',
  'GACCt',
  'GACCTTCGCAAATGCGTCCTGTGACt',
  'GACCTTCGCAAATGCGTCCTGTGACTGGTAGCCCACGCACCGGATTGAAACCAGTCGCCAGGAGGAACGAGATTCAACCTACTATAt'],
 'C': ['GACCTTCGCAAATGCGTCCTGTGACTGGTAGc',
  'GACCTTCGCAAATGCGTCCTGTGACTGGTAGCCCACGCACCGGATTGAAACCAGTCGCCAGGAGGAACGAGATTCAACCTACTATATATTGTACCAATTC',
  'GACc',
  'GACCTTCGCAAATGCGTCCTGTGACTGGTAGCCCACGCACCGGATTGAAACCAGTCGCCAGGAGGAACGAGATTCAAc',
  'GACCTTCGCAAATGCGTCc',
  'GACCTTCGc',
  'GACCTTCGCAAATGCGTCCTGTGAc',
  'GACCTTCGCAAATGCGT

dictionary elements can be retrieved using a key:

In [21]:
seq_run['A']

['GACCTTCGCAAa',
 'GACCTTCGCAAATGCGTCCTGTGACTGGTAGCCCa',
 'Ga',
 'GACCTTCGCAAATGCGTCCTGTGACTGGTa',
 'GACCTTCGCAAATGCGTCCTGTGACTGGTAGCCCACGCACCGGATTGa',
 'GACCTTCGCAAATGCGTCCTGTGACTGGTAGCCCACGCACCGGATTGAAACCAGTCGCCAGGAGGAACGa',
 'GACCTTCGCa',
 'GACCTTCGCAa',
 'Ga',
 'Ga']

## Drawing a sequencing gel

Now that we can simulate and store newly synthesized sequencing strands terminated with ddNTPs, let us try to draw a realistic representation of the sequencing gel. For this we will use several components that will be discussed in much greater detail in the upcoming lectures. These components are:

- [`pandas`](https://pandas.pydata.org/) - a dataframe manipulation tool
- [`altair`](https://altair-viz.github.io/) - a statistical visualization library built on top of the `vega-lite` visualization grammar

These two libraries will be used in almost all lectures concerning Python in this class.

[Gel electrophoresis](https://en.wikipedia.org/wiki/Gel_electrophoresis) separates molecules based on mass, shape, or charge. In the case of DNA, all molecules are universally negatively charged and thus will always migrate toward the (+) electrode. All our molecules are linear single-stranded pieces (our gel is *denaturing*), and so the only physical/chemical characteristic that distinguishes them is *length*. Therefore, the first thing we will do is convert our sequences into their lengths. For this we will initialize a new dictionary called `seq_lengths`:

In [22]:
seq_lengths = {'base':[],'length':[]}
for key in seq_run.keys():
    for sequence in seq_run[key]:
        seq_lengths['base'].append(key)
        seq_lengths['length'].append(len(sequence))

In [23]:
seq_lengths

{'base': ['A',
  'A',
  'A',
  'A',
  'A',
  'A',
  'A',
  'A',
  'A',
  'A',
  'T',
  'T',
  'T',
  'T',
  'T',
  'T',
  'T',
  'T',
  'T',
  'T',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'G',
  'G',
  'G',
  'G',
  'G',
  'G',
  'G',
  'G',
  'G',
  'G'],
 'length': [12,
  35,
  2,
  30,
  48,
  70,
  10,
  11,
  2,
  2,
  5,
  22,
  26,
  22,
  100,
  13,
  6,
  5,
  26,
  87,
  32,
  100,
  4,
  78,
  19,
  9,
  25,
  36,
  40,
  36,
  62,
  8,
  100,
  21,
  100,
  54,
  1,
  23,
  21,
  23]}

now let's import `pandas`:

In [24]:
import pandas as pd

and inject `seq_lengths` into a pandas *dataframe*:

In [25]:
sequences = pd.DataFrame(seq_lengths)

it looks pretty:

In [26]:
sequences

,base,length
0,A,12
1,A,35
2,A,2
3,A,30
4,A,48
5,A,70
6,A,10
7,A,11
8,A,2
9,A,2


In our data there is a number of DNA fragments that have identical length (just look at the dataframe above). We can condense these by grouping dataframe entries first by nucleotide (`['base']`) and then by length (`['length']`). For each group we will then compute `count` and put it into a new column named, unsurprisngly, `count`:

In [27]:
sequences_grouped_by_length = sequences.groupby(
    ['base','length']
).agg(
    count=pd.NamedAgg(
        column='length',
        aggfunc='count'
    )
).reset_index()

In [28]:
sequences_grouped_by_length

,base,length,count
0,A,2,3
1,A,10,1
2,A,11,1
3,A,12,1
4,A,30,1
5,A,35,1
6,A,48,1
7,A,70,1
8,C,4,1
9,C,9,1


The following chart is created using the `alt.Chart()` function and passing the data as an argument. The `mark_tick()` function is used to create a tick chart with a thickness of 4 pixels.

The chart is encoded with two main axis:

- y-axis which represents the length of the data and it is encoded by the `'length'` field of the data.
- x-axis which represents the base of the data and it is encoded by the `'base'` field of the data.
The chart also encodes a color, it encodes the `'count'` field of the data and it sets the legend to `None` and it uses the `'greys'` scale from the Altair library.

Finally, the chart properties are set to a width of 100 pixels and a height of 800 pixels.

In [29]:
import altair as alt
alt.Chart(sequences_grouped_by_length).mark_tick(thickness=4).encode(
    y = alt.Y('length:Q'),
    x = alt.X('base'),
    color=alt.Color('count:Q',legend=None,
                    scale=alt.Scale(scheme="greys"))
).properties(
    width=100,
    height=800)

alt.Chart(...)

And here is a color version of the same graph using just one line of the gel:

In [ ]:
import altair as alt
alt.Chart(sequences_grouped_by_length).mark_tick(thickness=4).encode(
    y = alt.Y('length:Q'),
    color=alt.Color('base:N',#legend=None,
                    scale=alt.Scale(scheme="set1"))
).properties(
    width=20,
    height=800)

In [ ]:
# Generate random sequences

seq = ''
for _ in range(300):
    seq += random.choice('ATCG')

## Putting everything together

In [ ]:
seq

In [ ]:
# Function simulating a single run of a single polymerase molecule

def ddN(number_of_iterations, template, base, ddN_ratio):
    new_strands = []
    for _ in range(number_of_iterations):
        synthesized_strand = ''
        for nucleotide in template:
            if nucleotide == base and random.random() > ddN_ratio:
                synthesized_strand += nucleotide.lower()
                break
            synthesized_strand += nucleotide
        new_strands.append(synthesized_strand)
    return(new_strands)

In [ ]:
# Generating simulated sequencing run

seq_run = {}
for nt in 'ATCG':
    seq_run[nt] = ddN(100000,seq,nt,0.95)

In [ ]:
# Computing lengths

seq_lengths = {'base':[],'length':[]}
for key in seq_run.keys():
    for sequence in seq_run[key]:
        seq_lengths['base'].append(key)
        seq_lengths['length'].append(len(sequence))

In [ ]:
# Converting dictionary into Pandas dataframe

sequences = pd.DataFrame(seq_lengths)

In [ ]:
# Grouping by nucleotide and length

sequences_grouped_by_length = sequences.groupby(
    ['base','length']
).agg(
    count=pd.NamedAgg(
        column='length',
        aggfunc='count'
    )
).reset_index()

In [ ]:
# Plotting (note the quadratic scale for realism)

import altair as alt
alt.Chart(sequences_grouped_by_length).mark_tick(thickness=4).encode(
    y = alt.Y('length:Q',scale=alt.Scale(type='sqrt')),
    x = alt.X('base'),
    color=alt.Color('count:Q',legend=None,
                    scale=alt.Scale(type='log',scheme="greys")),
    tooltip='count:Q'
).properties(
    width=100,
    height=800)

In [ ]:
# Plotting using color

import altair as alt
alt.Chart(sequences_grouped_by_length).mark_tick(thickness=4).encode(
    y = alt.Y('length:Q',scale=alt.Scale(type="sqrt")),
    color=alt.Color('base:N',#legend=None,
                    scale=alt.Scale(scheme="set1")),
    opacity=alt.Opacity('count:N',legend=None),
    tooltip='count:Q'
).properties(
    width=20,
    height=800)

## Homework

----

Chapters [1](https://colab.research.google.com/github/AllenDowney/ThinkPython/blob/v3/chapters/chap01.ipynb), [2](https://colab.research.google.com/github/AllenDowney/ThinkPython/blob/v3/chapters/chap02.ipynb), [3](https://colab.research.google.com/github/AllenDowney/ThinkPython/blob/v3/chapters/chap03.ipynb) from "Think Python"